In [1]:
from sklearn.datasets import make_classification

# Tabular Data

Tabular data is the data that you most often see. It is data that you can cleanly write in a table. It has a set number of rows and columns, and for our example below, all the data is numeric.

This is the one type of data that we will go over that is not necessarily suited to neural networks. Because it is so simple and so well studied, traditional ML can do quite well on it. 

That being said it makes a nice springboard to begin the rest of the tutorial.

To make this data we will be using sklearn `make_classification`. This will generate a dummy classification dataset:

In [32]:
dataset = make_classification(n_samples=10_000, n_features=20, n_classes=2)
x, y = dataset

In [33]:
x, y

(array([[ 1.43096299, -0.76448651, -0.14603622, ...,  1.25647633,
          0.95220237, -0.86957962],
        [-2.19153189,  1.60405674,  2.62852704, ..., -1.63368447,
          0.06884142, -0.67151764],
        [ 1.25853375, -0.67072596, -0.32322832, ..., -0.73368536,
         -0.64945835, -0.78647951],
        ...,
        [ 0.71661685,  1.36375015,  0.03613625, ...,  0.57719498,
          0.97163191, -0.33473533],
        [-1.37980976, -1.38402459,  0.84625386, ..., -0.87398062,
          0.76094365, -0.6931395 ],
        [ 0.20402275,  0.93395243, -2.28360233, ..., -0.30728972,
          1.16765538, -1.37796527]]), array([1, 1, 1, ..., 1, 1, 0]))

Because we have two classes, this is binary classification, so predicting either 0 or a 1 based off of these 20 features.

So now that we have the data we can just throw it into a NN right? 

Well not quite yet. Because a NN is basically a linear ML alg, we first need to scale all the inputs:

In [34]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

standardized_x = ss.fit_transform(x)

Perfect, now we can just throw it into a NN :) 

Yup for this data there is not too much else to it but to build the NN.

In [35]:
import tensorflow as tf

# dropout probability
p = .1

We are going to be using keras to build our NN. Because this is tabular data we can follow a fairly simple structure of a NN:

1. Standardize/Normalize
2. (Optional) Regularize/Dropout
3. Apply a Dense Layer

Let me talk about the first and the last.

Standardizing is important because of the way that NNs train by using gradient descent. If a particular layer's input is too big, then the gradients might be massive and the training process goes out of wack. 

The dense layer is the core of the NN and applies a non-linear transformation to the inputs allowing the NN to represent any non-linear function - or something like that. Regardless without that you couldn't learn.

Dropout is a simple way of regularizing NNs. The reason I put this as optional, is that there is some debate on whether you need dropout in addition to batch normalization.

Ultimately you can experiment with the amt of dropout you need in your network, and if it's none, so be it.

---

So all that being said below is our first NN.

In [36]:
inputs = tf.keras.layers.Input((20,), name='numeric_inputs')

In [37]:
x = tf.keras.layers.Dropout(p)(inputs)
x = tf.keras.layers.Dense(100, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(20, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(10, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
out = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(x)

Now there are probably a couple of questions as to the above:

* Why so many layers?
* Why so many neurons in each layer

Well a good rule of thumb is that your NN can have as many params as the number of data points that you have, and the above NN has half as many, so we could probably increase the number of parameters. 

As for the width vs the depth of the network, well there has been a ton of results on either side of the aisle and honeslty I'm not sure what to tell you other than experimentation.

Some things you might want to keep in mind are:

* Skip connections seem to be pretty cool
* Alternating small and large layers might be a thing too

In [21]:
model = tf.keras.models.Model(inputs=inputs, outputs=out)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [22]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
numeric_inputs (InputLayer)  [(None, 20)]              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               2100      
_________________________________________________________________
batch_normalization_v2_3 (Ba (None, 100)               400       
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 20)                2020      
_________________________________________________________________
batch_normalization_v2_4 (Ba (None, 20)                80  

As a final amendment to our data, I always like to use keras's `fit_generator` function, so I will often make a generator to feed data to the NN instead of using the default fit funtion.

In [38]:
import numpy as np

def bootstrap_sample_generator(batch_size):
    while True:
        batch_idx = np.random.choice(
            standardized_x.shape[0], batch_size)
        yield ({'numeric_inputs': standardized_x[batch_idx]}, 
               {'output': y[batch_idx]})

In [25]:
batch_size = 32

model.fit_generator(
    bootstrap_sample_generator(batch_size),
    steps_per_epoch=10_000 // batch_size,
    epochs=5,
    max_queue_size=10,
)

Epoch 1/5
313/312 [==============================] - 1s 2ms/step - loss: 0.2852 - accuracy: 0.8886
Epoch 2/5
313/312 [==============================] - 1s 2ms/step - loss: 0.2176 - accuracy: 0.9125
Epoch 3/5
313/312 [==============================] - 1s 2ms/step - loss: 0.2343 - accuracy: 0.9028
Epoch 4/5
313/312 [==============================] - 1s 2ms/step - loss: 0.2105 - accuracy: 0.9113
Epoch 5/5
313/312 [==============================] - 1s 2ms/step - loss: 0.2131 - accuracy: 0.9135
